<텍스트 전처리 과정>
* 클렌징
* 대/소문자 변경
* 특수문장 삭제
* 단어 등의 토큰화 작업
* 의미없는 단어 제거
* 어근 추출

등 텍스트 정규화 작업

이 후에 피처벡터화

민원데이터(2016~2019)와 주민참여예산 사업 간 유사도 파악
-> bert 모델에 인풋으로 함께 넣어줌 

# 라이브러리 임포트

In [1]:
import re
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.dates as mdates

from wordcloud import WordCloud
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import LatentDirichletAllocation
from nltk import word_tokenize, sent_tokenize

In [2]:
import warnings
import nltk
from nltk.corpus import stopwords
warnings.filterwarnings('ignore')

In [3]:
from nltk.stem import WordNetLemmatizer
import string

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [4]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

# 데이터 불러오기

In [5]:
seo_data_2010 = pd.read_excel('빅데이터분석 인턴쉽/20201029_세부사업별 세출현황, 분야별 세출현황, 재정자립도,자주도/01.세부사업별세출현황/2010년 세부사업별세출현황_서울시 및 산하기관.xls', header = 2)
seo_data_2011 = pd.read_excel('빅데이터분석 인턴쉽/20201029_세부사업별 세출현황, 분야별 세출현황, 재정자립도,자주도/01.세부사업별세출현황/2011년 세부사업별세출현황_서울시 및 산하기관.xls', header = 2)
seo_data_2012 = pd.read_excel('빅데이터분석 인턴쉽/20201029_세부사업별 세출현황, 분야별 세출현황, 재정자립도,자주도/01.세부사업별세출현황/2012년 세부사업별세출현황_서울시 및 산하기관.xls', header = 2)
seo_data_2013 = pd.read_excel('빅데이터분석 인턴쉽/20201029_세부사업별 세출현황, 분야별 세출현황, 재정자립도,자주도/01.세부사업별세출현황/2013년 세부사업별세출현황_서울시 및 산하기관.xls', header = 2)
seo_data_2014 = pd.read_excel('빅데이터분석 인턴쉽/20201029_세부사업별 세출현황, 분야별 세출현황, 재정자립도,자주도/01.세부사업별세출현황/2014년 세부사업별세출현황_서울시 및 산하기관.xls', header = 2)
seo_data_2015 = pd.read_excel('빅데이터분석 인턴쉽/20201029_세부사업별 세출현황, 분야별 세출현황, 재정자립도,자주도/01.세부사업별세출현황/2015년 세부사업별세출현황_서울시 및 산하기관.xls', header = 2)
seo_data_2016 = pd.read_excel('빅데이터분석 인턴쉽/20201029_세부사업별 세출현황, 분야별 세출현황, 재정자립도,자주도/01.세부사업별세출현황/2016년 세부사업별세출현황_서울시 및 산하기관.xls', header = 2)
seo_data_2017 = pd.read_excel('빅데이터분석 인턴쉽/20201029_세부사업별 세출현황, 분야별 세출현황, 재정자립도,자주도/01.세부사업별세출현황/2017년 세부사업별세출현황_서울시 및 산하기관.xls', header = 2)
seo_data_2018 = pd.read_excel('빅데이터분석 인턴쉽/20201029_세부사업별 세출현황, 분야별 세출현황, 재정자립도,자주도/01.세부사업별세출현황/2018년 세부사업별세출현황_서울시 및 산하기관.xls', header = 2)
seo_data_2019 = pd.read_excel('빅데이터분석 인턴쉽/20201029_세부사업별 세출현황, 분야별 세출현황, 재정자립도,자주도/01.세부사업별세출현황/2019년 세부사업별세출현황_서울시 및 산하기관.xls', header = 2)
seo_data_2020 = pd.read_excel('빅데이터분석 인턴쉽/20201029_세부사업별 세출현황, 분야별 세출현황, 재정자립도,자주도/01.세부사업별세출현황/2020년 세부사업별세출현황_서울시 및 산하기관.xls', header = 2)

bu_data_2010 = pd.read_excel('빅데이터분석 인턴쉽/20201029_세부사업별 세출현황, 분야별 세출현황, 재정자립도,자주도/01.세부사업별세출현황/2010년 세부사업별세출현황_부산광역시 및 산하기관.xls', header = 2)
bu_data_2011 = pd.read_excel('빅데이터분석 인턴쉽/20201029_세부사업별 세출현황, 분야별 세출현황, 재정자립도,자주도/01.세부사업별세출현황/2011년 세부사업별세출현황_부산광역시 및 산하기관.xls', header = 2)
bu_data_2012 = pd.read_excel('빅데이터분석 인턴쉽/20201029_세부사업별 세출현황, 분야별 세출현황, 재정자립도,자주도/01.세부사업별세출현황/2012년 세부사업별세출현황_부산광역시 및 산하기관.xls', header = 2)
bu_data_2013 = pd.read_excel('빅데이터분석 인턴쉽/20201029_세부사업별 세출현황, 분야별 세출현황, 재정자립도,자주도/01.세부사업별세출현황/2013년 세부사업별세출현황_부산광역시 및 산하기관.xls', header = 2)
bu_data_2014 = pd.read_excel('빅데이터분석 인턴쉽/20201029_세부사업별 세출현황, 분야별 세출현황, 재정자립도,자주도/01.세부사업별세출현황/2014년 세부사업별세출현황_부산광역시 및 산하기관.xls', header = 2)
bu_data_2015 = pd.read_excel('빅데이터분석 인턴쉽/20201029_세부사업별 세출현황, 분야별 세출현황, 재정자립도,자주도/01.세부사업별세출현황/2015년 세부사업별세출현황_부산광역시 및 산하기관.xls', header = 2)
bu_data_2016 = pd.read_excel('빅데이터분석 인턴쉽/20201029_세부사업별 세출현황, 분야별 세출현황, 재정자립도,자주도/01.세부사업별세출현황/2016년 세부사업별세출현황_부산광역시 및 산하기관.xls', header = 2)
bu_data_2017 = pd.read_excel('빅데이터분석 인턴쉽/20201029_세부사업별 세출현황, 분야별 세출현황, 재정자립도,자주도/01.세부사업별세출현황/2017년 세부사업별세출현황_부산광역시 및 산하기관.xls', header = 2)
bu_data_2018 = pd.read_excel('빅데이터분석 인턴쉽/20201029_세부사업별 세출현황, 분야별 세출현황, 재정자립도,자주도/01.세부사업별세출현황/2018년 세부사업별세출현황_부산광역시 및 산하기관.xls', header = 2)
bu_data_2019 = pd.read_excel('빅데이터분석 인턴쉽/20201029_세부사업별 세출현황, 분야별 세출현황, 재정자립도,자주도/01.세부사업별세출현황/2019년 세부사업별세출현황_부산광역시 및 산하기관.xls', header = 2)
bu_data_2020 = pd.read_excel('빅데이터분석 인턴쉽/20201029_세부사업별 세출현황, 분야별 세출현황, 재정자립도,자주도/01.세부사업별세출현황/2020년 세부사업별세출현황_부산광역시 및 산하기관.xls', header = 2)

## 데이터 합치기

In [6]:
seo_data = seo_data_2010.append(seo_data_2011)
seo_data = seo_data.append(seo_data_2012)
seo_data = seo_data.append(seo_data_2013)
seo_data = seo_data.append(seo_data_2014)
seo_data = seo_data.append(seo_data_2015)
seo_data = seo_data.append(seo_data_2016)
seo_data = seo_data.append(seo_data_2017)
seo_data = seo_data.append(seo_data_2018)
seo_data = seo_data.append(seo_data_2019)
seo_data = seo_data.append(seo_data_2020)
    
bu_data = bu_data_2010.append(bu_data_2011)
bu_data = bu_data.append(bu_data_2012)
bu_data = bu_data.append(bu_data_2013)
bu_data = bu_data.append(bu_data_2014)
bu_data = bu_data.append(bu_data_2015)
bu_data = bu_data.append(bu_data_2016)
bu_data = bu_data.append(bu_data_2017)
bu_data = bu_data.append(bu_data_2018)
bu_data = bu_data.append(bu_data_2019)
bu_data = bu_data.append(bu_data_2020)

## 데이터 볼륨 줄이기
### 특정 사업 제거

In [7]:
jeong_list = list(seo_data.정책사업명.unique())
com_j = []
for i in jeong_list:
    if ('행정운영경비'  in i) or ('재무활동' in i) or ('일반예산' in i) :
        com_j.append(i)

In [8]:
com_j

['일반예산(타기관지원등)',
 '재무활동',
 '재무활동(재무국 재무과)',
 '일반예산(재무활동)',
 '재무활동(경쟁력강화본부 관광진흥담당관)',
 '재무활동(맑은환경본부 기후변화담당관)',
 '재무활동(여성가족정책관 저출산대책담당관)',
 '재무활동(경쟁력강화본부 일자리정책담당관)',
 '재무활동(주택국 주택공급과)',
 '재무활동(여성가족정책관 건강증진담당관)',
 '재무활동(복지국 식품안전과)',
 '재무활동(경제진흥본부 창업소상공인과)',
 '일반예산(타기관 지원 등)',
 '일반예산(예비비)',
 '일반예산(행정운영경비)',
 '행정운영경비',
 '행정운영경비(시립미술관 총무과)',
 '행정운영경비(경영기획실 학교지원담당관)',
 '행정운영경비(여성가족정책관 건강증진담당관)',
 '행정운영경비(균형발전본부 도심재정비1담당관)',
 '행정운영경비(균형발전본부 도심재정비2담당관)',
 '행정운영경비(균형발전본부 뉴타운사업2담당관)',
 '행정운영경비(여성가족정책관 저출산대책담당관)',
 '행정운영경비(주택국 신청사건립과)',
 '행정운영경비(문화국 문화예술과)',
 '행정운영경비(균형발전본부 동남권유통단지조성담당관)',
 '행정운영경비(도시계획국 마곡개발과)',
 '행정운영경비(경영기획실 평생교육담당관)',
 '행정운영경비(디자인서울총괄본부 디자인기획담당관)',
 '행정운영경비(주택국 주택공급과)',
 '행정운영경비(푸른도시국 푸른도시정책과)',
 '행정운영경비(경영기획실 정책비전담당관)',
 '행정운영경비(디자인서울총괄본부 WDC담당관)',
 '행정운영경비(행정국 특별사법경찰지원과)',
 '행정운영경비(균형발전본부 동대문디자인파크담당관)',
 '행정운영경비(복지국 원산지관리과)',
 '행정운영경비(한옥문화과)',
 '행정운영경비(주택국 주택정책개발반)',
 '행정운영경비(문화국 한성백제박물관건립추진단)',
 '행정운영경비(경쟁력강화본부 일자리지원담당관)',
 '행정운영경비(문화국 대표도서관건립추진반)',
 '재무활동(도시기반시설본부 도시철도국)',

In [9]:
jeong_listb = list(bu_data.정책사업명.unique())
com_jb = []
for i in jeong_listb:
    if ('행정운영경비'  in i) or ('재무활동' in i) or ('일반예산' in i) :
        com_jb.append(i)

In [10]:
com_jb

['재무활동(인재개발원)',
 '재무활동(예산담당관)',
 '재무활동(자치행정과)',
 '재무활동(도시재생과)',
 '재무활동(재난안전담당관)',
 '재무활동(예방대응과)',
 '재무활동(문화예술과)',
 '재무활동(영상문화산업과)',
 '재무활동(체육진흥과)',
 '재무활동(환경보전과)',
 '재무활동(환경정책과)',
 '재무활동(사회복지과)',
 '재무활동(여성정책담당관)',
 '재무활동(여성회관)',
 '재무활동(아동보호종합센터)',
 '재무활동(고령화대책과)',
 '재무활동(보건환경연구원)',
 '재무활동(건강증진과)',
 '재무활동(농축산유통과)',
 '재무활동(농업기술센터)',
 '재무활동(수산진흥과)',
 '재무활동(수산자원연구소)',
 '재무활동(교육협력과)',
 '재무활동(국제협력과)',
 '재무활동(투자유치과)',
 '재무활동(기업지원과)',
 '재무활동(경제정책과)',
 '재무활동(도로계획담당관)',
 '재무활동(교통정책과-도로)',
 '재무활동(교통정책과)',
 '재무활동(낙동강살리기사업부)',
 '재무활동(건설정책담당관)',
 '재무활동(도시정비담당관)',
 '재무활동(관광단지추진단)',
 '재무활동(시설계획과)',
 '재무활동(시민공원추진단)',
 '재무활동(혁신도시개발단)',
 '재무활동(건설본부)',
 '행정운영경비(교통정책과)',
 '행정운영경비(중부소방서)',
 '행정운영경비(소방행정과)',
 '행정운영경비(자치행정과)',
 '행정운영경비(문화예술과)',
 '행정운영경비(물관리과)',
 '행정운영경비(환경자원공원사업소)',
 '행정운영경비(교통관리과)',
 '행정운영경비(회계재산담당관)',
 '행정운영경비(여성회관)',
 '행정운영경비(충렬사관리사무소)',
 '행정운영경비(반여농산물도매시장관리사업소)',
 '행정운영경비(건설안전시험사업소)',
 '행정운영경비(시립박물관)',
 '행정운영경비(시민봉사과)',
 '행정운영경비(서울사무소)',
 '행정운영경비(건설정책담당관)',
 '행정운영경비(금련산청소년수련원)',
 '행정운영경비(

In [11]:
seo_data['재무여부'] = seo_data['정책사업명'].apply(lambda x: 1 if x in com_j else 0)
bu_data['재무여부'] = bu_data['정책사업명'].apply(lambda x: 1 if x in com_jb else 0)

In [12]:
s_data = seo_data[seo_data['재무여부']==0]
b_data = bu_data[bu_data['재무여부']==0]

In [13]:
b_data.head()

,연도,정렬,지역,자치단체,회계구분,분야,부문,정책사업코드,정책사업명,단위사업코드,...,공약사항,국가지방계획,부처명,국가직접사업,주민참여예산,일자리사업,일자리사업구분,행사축제성사업,균특지방전환사업,재무여부
0,2010,21200,부산광역시,부산광역시,일반회계,일반공공행정,입법및선거관리,6260000200710138,지방의회 운영 지원,6260000200720370,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2010,21200,부산광역시,부산광역시,일반회계,일반공공행정,입법및선거관리,6260000200710138,지방의회 운영 지원,6260000200720370,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,2010,21200,부산광역시,부산광역시,일반회계,일반공공행정,입법및선거관리,6260000200710138,지방의회 운영 지원,6260000200720370,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,2010,21200,부산광역시,부산광역시,일반회계,일반공공행정,입법및선거관리,6260000200710138,지방의회 운영 지원,6260000200720370,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,2010,21200,부산광역시,부산광역시,일반회계,일반공공행정,입법및선거관리,6260000200710138,지방의회 운영 지원,6260000200720370,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


### 주민참여예산과 관련없는 부문 제거

In [14]:
seo_data[seo_data['주민참여예산'].isnull()].부문.value_counts()

일반행정          32816
보건의료          23618
지역및도시         22276
기타            17751
보육ㆍ가족및여성      14445
노인ㆍ청소년        13709
취약계층지원        13135
대중교통ㆍ물류등기타     9627
문화예술           9234
도로             8915
폐기물            6347
기초생활보장         5821
재난방재ㆍ민방위       5786
체육             5401
산업진흥ㆍ고도화       4728
유아및초중등교육       4079
노동             4031
환경보호일반         3911
상하수도ㆍ수질        3712
지방행정ㆍ재정지원      3153
식품의약안전         3148
수자원            2859
자연             2367
농업ㆍ농촌          2014
입법및선거관리        1984
산업ㆍ중소기업일반      1946
문화재            1907
사회복지일반         1898
평생ㆍ직업교육        1883
관광             1548
대기             1224
임업ㆍ산촌          1211
주택              855
문화및관광일반         762
산업금융지원          680
에너지및자원개발        658
보훈              469
소방              377
예비비             337
무역및투자유치         322
재정ㆍ금융           238
도시철도            195
고등교육            106
산업단지             58
과학기술일반           50
경찰               47
산업기술지원           45
Name: 부문, dtype: int64

In [15]:
bu_data[bu_data['주민참여예산'].notnull()].부문.unique()

array(['식품의약안전', '도로', '일반행정', '수자원', '지역및도시', '대중교통ㆍ물류등기타', '보건의료',
       '상하수도ㆍ수질', '관광', '농업ㆍ농촌', '사회복지일반', '취약계층지원', '재난방재ㆍ민방위',
       '평생ㆍ직업교육', '지방행정ㆍ재정지원', '체육', '폐기물', '환경보호일반', '임업ㆍ산촌', '노인ㆍ청소년'],
      dtype=object)

In [16]:
set(bu_data.부문.unique())-set(seo_data.부문.unique()) 

{'과학기술연구지원', '항공ㆍ공항', '해양', '해양수산ㆍ어촌', '해운ㆍ항만'}

In [17]:
bumun_s = list(seo_data[seo_data['주민참여예산'].notnull()].부문.unique())
bumun_b = list(bu_data[bu_data['주민참여예산'].notnull()].부문.unique())

In [18]:
s_data['부문여부'] = s_data['부문'].apply(lambda x: 1 if x in bumun_s else 0)
b_data['부문여부'] = b_data['부문'].apply(lambda x: 1 if x in bumun_b else 0)

In [19]:
s_data = s_data[s_data['부문여부']==1]
b_data = b_data[b_data['부문여부']==1]

In [20]:
s_data.부문.unique()

array(['재정ㆍ금융', '일반행정', '재난방재ㆍ민방위', '유아및초중등교육', '평생ㆍ직업교육', '문화예술', '관광',
       '체육', '문화재', '문화및관광일반', '상하수도ㆍ수질', '폐기물', '대기', '자연', '환경보호일반',
       '취약계층지원', '보육ㆍ가족및여성', '노인ㆍ청소년', '노동', '주택', '보건의료', '식품의약안전',
       '농업ㆍ농촌', '산업금융지원', '산업진흥ㆍ고도화', '산업ㆍ중소기업일반', '도로', '대중교통ㆍ물류등기타',
       '수자원', '지역및도시', '기초생활보장', '임업ㆍ산촌', '에너지및자원개발', '사회복지일반', '소방'],
      dtype=object)

In [21]:
b_data.부문.unique()

array(['지방행정ㆍ재정지원', '일반행정', '재난방재ㆍ민방위', '평생ㆍ직업교육', '관광', '체육', '상하수도ㆍ수질',
       '폐기물', '환경보호일반', '취약계층지원', '노인ㆍ청소년', '보건의료', '식품의약안전', '농업ㆍ농촌',
       '임업ㆍ산촌', '도로', '대중교통ㆍ물류등기타', '수자원', '지역및도시', '사회복지일반'],
      dtype=object)

In [22]:
b_data[b_data['주민참여예산'].notnull()].정책사업명.unique()

array(['식의약품 안전관리', '도로 계획 및 건설', '주민자치 역량강화', '도시 및 기반시설 구축',
       '하수 및 소하천 관리', '도시기반시설 확충', '산복도로 르네상스 프로젝트 추진', '주차질서확립', '건강증진',
       '도로시설 확충', '공원녹지조성 및 관리', '지방도 건설 확포장', '효율적인 도로관리 운영',
       '하수기반시설 관리', '안전하고 쾌적한 교통환경 조성', '관광특구에 걸맞는 쾌적한 공원 관리', '교통 안전 확보',
       '주차질서 확립', '질높은 보건의료서비스 제공', '도시 녹지공간 확충', '농업 경쟁력 강화', '하천 정비',
       '도시개발', '맵시있는 도시디자인', '도시 기반시설 구축 및 관리', '지역사회복지서비스 지원',
       '연산4동 사업경비', '도로시설관리', '취약계층지원', '재해 및 재난 복구능력 강화',
       '협치행정의 기반강화 및 시민참여 활성화', '주민자치 행정지원 역량강화', '평생학습 체제 구축 및 인재육성',
       '도시재생 활성화', '도시 및 주거환경정비', '행정복지센터운영활성화(범일1동)', '주민센터운영활성화(초량2동)',
       '주민센터운영활성화(범일2동)', '주민센터운영 활성화(수정5동)', '주민센터 운영활성화(수정4동)',
       '주민센터 운영 활성화(수정2동)', '주민센터 운영 활성화(초량3동)', '주민센터운영활성화(좌천동)',
       '주민자치 기반강화(전포2동)', '주민자치 기반강화(당감4동)', '예산운용', '도로시설 관리', '교통행정 개선',
       '주민자치 및 행정복지 기반강화(대연3동)', '평생학습체제 구축', '관광기반 확충 및 관리', '체육진흥',
       '청소관리', '자연환경보전', '푸른 남구 가꾸기', '도로유지관리', '안전하고 쾌적한 도로환경 조성',
       '선진 청소서비스 제공', '지방도 건설 확포장 및 건설사

In [24]:
seo_data.describe()

,연도,정렬,정책사업코드,단위사업코드,합계,국비,시도비,시군구비,기타,실집행액,집행잔액,재무여부
count,243358.000000,243358.000000,2.433580e+05,2.433580e+05,2.433580e+05,2.433580e+05,2.433580e+05,2.433580e+05,2.433580e+05,2.433580e+05,2.433580e+05,243358.000000
mean,2015.885814,11450.842006,3.570671e+15,3.570671e+15,2.058028e+06,3.245811e+05,1.316277e+06,4.009820e+05,1.618720e+04,1.729435e+06,3.285929e+05,0.121175
std,3.097642,147.879240,1.071314e+15,1.071314e+15,3.063204e+07,1.033178e+07,2.723587e+07,3.157200e+06,1.223932e+06,2.909907e+07,5.982770e+06,0.326332
min,2010.000000,11100.000000,3.000000e+15,3.000000e+15,0.000000e+00,-1.000000e+03,-4.200000e+02,-3.000000e+04,0.000000e+00,0.000000e+00,-1.500000e+08,0.000000
25%,2013.000000,11503.000000,3.070000e+15,3.070000e+15,2.000000e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.108825e+04,3.030000e+02,0.000000
50%,2016.000000,11511.000000,3.140000e+15,3.140000e+15,9.550600e+04,0.000000e+00,0.000000e+00,2.000000e+04,0.000000e+00,6.070150e+04,5.350000e+03,0.000000
75%,2019.000000,11518.000000,3.220000e+15,3.220000e+15,4.000000e+05,0.000000e+00,5.432300e+04,1.343725e+05,0.000000e+00,2.722508e+05,4.194475e+04,0.000000
max,2020.000000,11525.000000,6.110000e+15,6.110000e+15,3.706210e+09,2.183317e+09,3.706210e+09,2.136911e+08,3.460000e+08,3.512656e+09,9.944618e+08,1.000000


In [25]:
s_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 207388 entries, 35 to 32384
Data columns (total 32 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   연도        207388 non-null  int64 
 1   정렬        207388 non-null  int64 
 2   지역        207388 non-null  object
 3   자치단체      207388 non-null  object
 4   회계구분      207388 non-null  object
 5   분야        207388 non-null  object
 6   부문        207388 non-null  object
 7   정책사업코드    207388 non-null  int64 
 8   정책사업명     207388 non-null  object
 9   단위사업코드    207388 non-null  int64 
 10  단위사업명     207388 non-null  object
 11  세부사업코드    207388 non-null  object
 12  세부사업명     207388 non-null  object
 13  합계        207388 non-null  int64 
 14  국비        207388 non-null  int64 
 15  시도비       207388 non-null  int64 
 16  시군구비      207388 non-null  int64 
 17  기타        207388 non-null  int64 
 18  실집행액      207388 non-null  int64 
 19  집행잔액      207388 non-null  int64 
 20  다기능       15121 non-null  

# 텍스트 데이터 전처리
## 정책사업명, 정책사업코드

In [30]:
len(s_data.정책사업명.unique())

2350

In [31]:
len(s_data.정책사업코드.unique())

2507

In [54]:
s_data['정책사업코드d'] = s_data['정책사업코드'].apply(lambda x: str(x)[:7])
s_data['정책사업코드b'] = s_data['정책사업코드'].apply(lambda x: str(x)[8:])

In [38]:
len(s_data['정책사업코드d'].unique()) #코드는 안 봐도 될듯

26

In [55]:
len(s_data['정책사업코드b'].unique())

925

In [56]:
len(s_data['정책사업명'].unique())

2350

In [60]:
ss = s_data[['정책사업코드b','정책사업명']].drop_duplicates() #3047
len(ss.정책사업명.unique()) #2350

2350

In [61]:
ss

,정책사업코드b,정책사업명
35,00811001,효율적 재정운영
44,00810001,전략적 시정홍보 강화
56,00810003,공정한 보도체계 확립
61,00810009,시정평가 기능 강화
63,00810011,맑고 깨끗한 공직풍토 조성
...,...,...
31094,01110016,재해 및 재난 예방
31286,00910033,민방위·재난안전 운영
31493,02010012,녹색에너지 도시 조성
31792,02010009,고용촉진 및 생활안정


In [57]:
s_data[['정책사업코드','정책사업코드b','정책사업명','단위사업명','세부사업명']].drop_duplicates()

,정책사업코드,정책사업코드b,정책사업명,단위사업명,세부사업명
35,6110000200811001,00811001,효율적 재정운영,재정운영의 효율성 제고,"재정정보(재정계획, 기금관리)시스템 운영"
36,6110000200811001,00811001,효율적 재정운영,재정운영의 효율성 제고,지방재정투융자사업 심사
37,6110000200811001,00811001,효율적 재정운영,재정운영의 효율성 제고,지역상생발전기금출연
38,6110000200811001,00811001,효율적 재정운영,투자출연기관 경영효율화,투자 · 출연기관 창의경영 추진
39,6110000200811001,00811001,효율적 재정운영,투자출연기관 경영효율화,투자출연기관 경영평가
...,...,...,...,...,...
32304,3240000200810039,00810039,안전하고 쾌적한 도시공간 조성,건축물 안전확보,예비비(건축안전특별회계)
32319,3240000200910037,00910037,주차시설확충 및 주차문화개선,주거지역 주차공간 확보,천호3동 노후공공청사 복합개발사업(공영주차장 확충)
32334,3240000200810054,00810054,더불어 행복한 도시경관 조성,무질서한 광고물 특별관리,대형 옥외광고물 수시 안전점검
32335,3240000200810054,00810054,더불어 행복한 도시경관 조성,광고문화 업그레이드,공공디자인사업


In [46]:
len(s_data[s_data['주민참여예산'].notnull()].정책사업명.unique())

388

In [53]:
name_j = list(s_data[s_data['주민참여예산'].notnull()].정책사업명.unique())
for i in name_j:
    if '운영' in i:
        print(i)    

정보인프라 확충 및 운영
한강공원 관리 및 운영
어르신복지 수준향상 및 장사시설 운영 내실화
도서관 설치 운영
박물관 운영의 활성화
식물종다양성 보전을 통한 세계적수준의 식물원 운영
효율적인 동 행정 운영
버스운영체계 개선 및 관리
중구민을 위한 동행정 운영(황학동)
중구민을 위한 동행정 운영(을지로동)
중구민을 위한 동행정 운영(다산동)
동행정 운영(약수동)
중구민을 위한 동행정 운영(중림동)
자활사업 운영


`재정`과 관련된 키워드는 주민참여예산의 사업명에 없음. 정책사업명의 토픽을 추출하여 주민참여 예산과 관련이 없는 토픽은 제거

In [47]:
len(s_data[s_data['주민참여예산'].notnull()].단위사업명.unique())

553

In [48]:
len(s_data[s_data['주민참여예산'].notnull()].세부사업명.unique())

1553

문장 단위 임베딩이 필요할 듯,,, 은근 정제가 잘 되어 있음.

### 정책사업 토큰화

In [91]:
s_data['정책사업토큰'] = s_data['정책사업명'].apply(lambda x: x.split(' '))

In [113]:
 s_data['정책사업명']

35           효율적 재정운영
36           효율적 재정운영
37           효율적 재정운영
38           효율적 재정운영
39           효율적 재정운영
             ...     
32372      학교교육 기반 구축
32374            청소관리
32382    재난대비 대응능력 향상
32383    재난대비 대응능력 향상
32384    재난대비 대응능력 향상
Name: 정책사업명, Length: 207388, dtype: object

In [112]:
s_data['정책사업토큰']

35            [효율적, 재정운영]
36            [효율적, 재정운영]
37            [효율적, 재정운영]
38            [효율적, 재정운영]
39            [효율적, 재정운영]
               ...       
32372      [학교교육, 기반, 구축]
32374              [청소관리]
32382    [재난대비, 대응능력, 향상]
32383    [재난대비, 대응능력, 향상]
32384    [재난대비, 대응능력, 향상]
Name: 정책사업토큰, Length: 207388, dtype: object

In [75]:
from konlpy.tag import Okt
from konlpy.tag import Mecab
from konlpy.tag import Kkma

In [65]:
mecab = Mecab(dicpath=r"C:\mecab\mecab-ko-dic")

In [109]:
stopwords = [',','의','와','및','·','으로','은','는','이','가','하','도','적','인','로','한','를','과','(',')','고','수','화','등','있']

In [116]:
tokenized_j = []
for sentence in s_data['정책사업명']:
    temp_X = mecab.morphs(sentence) #토큰화
    temp_X = [word for word in temp_X if not word in stopwords]
    tokenized_j.append(temp_X)

In [117]:
tokenized_j

[['효율', '재정', '운영'],
 ['효율', '재정', '운영'],
 ['효율', '재정', '운영'],
 ['효율', '재정', '운영'],
 ['효율', '재정', '운영'],
 ['효율', '재정', '운영'],
 ['효율', '재정', '운영'],
 ['효율', '재정', '운영'],
 ['전략', '시정', '홍보', '강화'],
 ['전략', '시정', '홍보', '강화'],
 ['전략', '시정', '홍보', '강화'],
 ['전략', '시정', '홍보', '강화'],
 ['전략', '시정', '홍보', '강화'],
 ['전략', '시정', '홍보', '강화'],
 ['전략', '시정', '홍보', '강화'],
 ['전략', '시정', '홍보', '강화'],
 ['전략', '시정', '홍보', '강화'],
 ['전략', '시정', '홍보', '강화'],
 ['전략', '시정', '홍보', '강화'],
 ['전략', '시정', '홍보', '강화'],
 ['공정', '보도', '체계', '확립'],
 ['공정', '보도', '체계', '확립'],
 ['공정', '보도', '체계', '확립'],
 ['공정', '보도', '체계', '확립'],
 ['공정', '보도', '체계', '확립'],
 ['시정', '평가', '기능', '강화'],
 ['시정', '평가', '기능', '강화'],
 ['맑', '깨끗', '공직', '풍토', '조성'],
 ['맑', '깨끗', '공직', '풍토', '조성'],
 ['적극', '민원', '처리', '시민', '고충', '민원', '처리', '수준', '향상'],
 ['적극', '민원', '처리', '시민', '고충', '민원', '처리', '수준', '향상'],
 ['적극', '민원', '처리', '시민', '고충', '민원', '처리', '수준', '향상'],
 ['적극', '민원', '처리', '시민', '고충', '민원', '처리', '수준', '향상'],
 ['서울', '브랜드', '해외', '마케팅']

In [103]:
mecab.morphs('전략적, 시정홍보강화')

['전략', '적', ',', '시정', '홍보', '강화']

토큰화를 하는 게 나은 건가 싶음;; 

In [99]:
s_data.reset_index().loc[37,'정책사업토큰']

['서울브랜드', '해외마케팅']

In [100]:
tokenized_j = []
for i in range(len(s_data['정책사업토큰'])):
    for sentence in s_data.reset_index().loc[i, '정책사업토큰']:
        temp_X = mecab.morphs(sentence) #토큰화
        #temp_X = [work for word in temp_X if not word in stopwords]
        tokenized_j[i].append(temp_X)
tokenized_j

IndexError: list index out of range